In [1]:
!pip install transformers vllm==0.6.0


[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
import string
from vllm import LLM, SamplingParams

# LLM
llm = LLM(model="iamjoon/Llama-3-ESG-SCORE-EVALUATOR")

INFO 11-19 06:32:20 llm_engine.py:213] Initializing an LLM engine (v0.6.0) with config: model='iamjoon/Llama-3-ESG-SCORE-EVALUATOR', speculative_config=None, tokenizer='iamjoon/Llama-3-ESG-SCORE-EVALUATOR', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=iamjoon/Llama-3-ESG-SCORE-EVALUATOR, use_v2_block_manager=False, num_scheduler_steps=1, enable_prefix_c

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 11-19 06:32:26 model_runner.py:926] Loading model weights took 14.9595 GB
INFO 11-19 06:32:27 gpu_executor.py:122] # GPU blocks: 27924, # CPU blocks: 2048
INFO 11-19 06:32:30 model_runner.py:1217] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 11-19 06:32:30 model_runner.py:1221] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 11-19 06:32:45 model_runner.py:1335] Graph capturing finished in 15 secs.


In [75]:
# 프롬프트 템플릿 준비
template = string.Template("""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

${system_content}<|eot_id|><|start_header_id|>user<|end_header_id|>

${user_content}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
""")

system_prompt = '''당신은 ESG에 대해서 점수를 계산하는 모델입니다. 평가 기준은 다음과 같습니다.

## 지시사항
주어진 평가 기준을 바탕으로 뉴스 기사로부터 환경, 사회, 지배 구조에 대해서 점수를 측정하십시오.

## 평가 기준
환경
1점: 환경 오염 유발, 법적 제재, 규제 위반 등의 사건이 발생한 경우. 예: 심각한 오염 사고, 규제 위반으로 인한 벌금 부과
2점: 환경 보호에 대한 소극적인 태도 또는 일회성 활동에 그치는 경우. 예: 임시적인 환경 캠페인 참여, 적극적인 감축 목표 미설정
3점: 환경 보호를 위한 노력이 있으나 체계적인 전략이 부족한 경우. 예: 온실가스 감축 계획 발표, 일부 친환경 인증 획득
4점: 체계적인 환경 관리 계획이 있으며, 목표 달성에 꾸준히 투자하는 경우. 예: 지속 가능한 에너지 사용, 체계적인 폐기물 관리 프로그램 운영
5점: 적극적인 환경 정책과 뛰어난 실적 결과를 보이며 업계 선도적 활동을 하는 경우. 예: 탄소 배출 네트 제로(Net Zero) 목표 실현, 친환경 기술 선도

사회
1점: 인권, 근로자 안전 문제 발생, 지역 사회와의 갈등 등 부정적 이슈 발생. 예: 직원 착취 문제, 안전 관리 소홀로 인한 사고
2점: 직원 복지나 지역 사회 기여에 대한 제한적 활동 또는 단기적인 참여. 예: 일회성 기부, 사내 복지 프로그램 부족
3점: 직원 복지 및 사회적 기여에 관심을 가지며 일부 프로그램을 운영하는 경우. 예: 사내 복지 프로그램 운영, 특정 지역 사회 활동 참여
4점: 직원 복지, 다양성, 지역 사회 공헌에 적극적으로 나서며 책임감 있게 운영하는 경우. 예: 다양성 강화 프로그램, 정기적인 사회공헌 활동
5점: 사회적 가치 창출에 지속적으로 기여하며 업계 모범적인 활동을 보여주는 경우. 예: 폭넓은 사회적 책임 활동, 소외 계층 지원 프로그램

지배구조
1점: 지배구조의 투명성 부족, 부패 사건 또는 규제 위반이 발생한 경우. 예: 부패 스캔들, 재무 조작
2점: 지배구조 개선 노력이 부족하며, 외부 감사나 투명성 문제에 있는 경우. 예: 의사 결정 과정 불투명, 주주와의 소통 부족
3점: 기본적인 지배구조 원칙을 따르나, 일부 개선의 여지가 있는 경우. 예: 주주와의 소통 강화 중, 투명성 개선 노력
4점: 투명하고 윤리적인 지배구조를 실천하고 있으며, 책임 있는 경영을 하는 경우. 예: 정기적인 외부 감사, 투명한 의사결정 구조
5점: 업계에서 모범이 되는 윤리적 지배구조를 유지하고 지속적 개선을 보이는 경우. 예: 독립적 이사회 구성, 주주 및 이해관계자와의 적극적 소통

## 답변 형식
답변 형식은 반드시 아래의 형식을 따르십시오. 근거:, E:, S:, G: 외에는 그 어떠한 문장도 덧붙이지 마십시오.

근거: 해당 뉴스 기사에 대해서 ESG 점수를 작성한 근거를 작성합니다.
E: 1~5 사이의 정수
S: 1~5 사이의 정수
G: 1~5 사이의 정수

## 뉴스 기사'''

user_prompt = '''월남쌈 대신 '비비고' 만두 먹고 CGV서 영화 봐요"…'베트남 젠지' 홀린 CJ CJ그룹은 지난 10일부터 9월 1일까지 베트남에서 K컬처 축제 'CJ K FESTA'를 연다.ⓒ 뉴스1/김진희 기자.  (호찌민=뉴스1) 김진희 기자 = "CJ(001040) 로고가 화려하고 예쁩니다. CGV를 통해 CJ를 알게 됐는데 젊고 역동적이어서 '젠지' 세대에 적합한 이미지라는 생각이 듭니다." -화이빠오(24·남)  지난 15~16일(현지시간) 베트남 호찌민 일대에서 열린 'CJ K FESTA'를 찾은 베트남 현지인들은 극찬을 쏟아냈다.  CJ그룹은 베트남에 진출한 계열사를 총동원해 CJ K FESTA를 올해 처음으로 개최했다. 이번 축제는 △K푸드 Week(8월 10~16일) △K스포츠 Week(8월 17~23일) △K무비 Week(8월 24일~9월 1일) 등 3가지 테마로 구성됐다.  CJ는 호찌민 소재 유명 마트와 쇼핑몰 내 CGV 매장에서 푸드쇼를 운영하며 브랜드 홍보에 적극 나섰다.  CJ그룹은 지난 10일부터 9월 1일까지 베트남에서 K컬처 축제 'CJ K FESTA'를 연다.ⓒ 뉴스1/김진희 기자.  15일 오후 호찌민 소재 메가마켓에는 더운 날씨임에도 마트 외부 공간에 사람들로 북적였다. CJ 제품을 선보이는 푸드트럭을 구경하기 위해 찾은 이들이었다. 한국과 달리 베트남에서는 푸드트럭이 생소하다고 한다.  이날 CJ는 CJ제일제당 비비고의 해산물 만두, 호떡, 김스낵과 뚜레쥬르의 빵을 선보였다. CJ제일제당의 자회사인 베트남 현지 냉동식품기업 '까우제'(Cautre)의 제품도 만나볼 수 있었다.  호찌민 시민 화이빠오는 "예전에 만두를 많이 먹어 봤지만 비비고 만두는 독특하고 맛있다"며 "특히 푸드트럭을 처음 봐서 신기한데, 이렇게 화려하게 시식·광고하는 건 처음 본다"고 말했다.  CJ그룹은 지난 10일부터 9월 1일까지 베트남에서 K컬처 축제 'CJ K FESTA'를 연다.ⓒ 뉴스1/김진희 기자.  아이와 함께 푸드트럭을 방문한 주부 흐엉(34·여)은 "거주하고 있는 아파트 로비에 뚜레쥬르 매장이 있어서 평소에도 많이 애용해 CJ를 알고 있었다"며 "비비고 만두, 김치는 물론 CGV 등 CJ 계열사를 평소 많이 애용하는 편"이라고 언급했다  메가마켓 내 푸드트럭 행사는 지난 11~15일 오후 4시부터 오후 9시 30분까지 운영됐는데 하루 방문객이 최대 1000명에 달했다고 한다.  CJ는 호찌민 내 재래시장, 대공원, 호수공원 등 총 10곳에서 푸드트럭을 운영했다.  CJ그룹은 지난 10일부터 9월 1일까지 베트남에서 K컬처 축제 'CJ K FESTA'를 연다.ⓒ 뉴스1/김진희 기자.  16일 이마트 고밥점에서 열린 쿠킹쇼 행사장에도 CJ 제품을 체험하기 위해 온 방문객들로 분주했다.  CJ는 단순히 자사 제품을 선보이는 것이 아니라 셰프, 스타 유튜버 등을 초청해 현지화된 음식을 제공해 눈길을 끌었다.  베트남에서 '한꿕 브로스'(Han Quoc Bros·한국 형제들) 채널을 운영, 100만 팔로워를 보유한 한국인 인기 유튜버 최종락 씨는 이날 셰프로 나섰다.  최 씨는 "CJ 비비고 만두, 짜조, 김과자를 베트남식으로 퓨전화해 선보였는데 현지 분들 반응이 너무 좋아 뿌듯했다"며 "한국인이 한국 음식을 소개하는데 너무 좋아해 주셔서 감사하다"고 밝혔다.  이날 행사장을 찾은 황 모 씨(57·남)는 "비비고 만두를 처음 먹어봤는데 베트남 음식이 아닌데도 잘 맞다"며 "비비고 만두가 베트남 유명 요리인 월남쌈을 대체할 수 있을 것 같다"고 엄지를 치켜세웠다.  그는 이 같은 행사 취지에 대해서도 "브랜딩 활동이 너무 신선하다"며 "CJ 디자인이 눈에 띄어서 굉장히 젊어 보인다"고 말했다.  CJ그룹은 지난 10일부터 9월 1일까지 베트남에서 K컬처 축제 'CJ K FESTA'를 연다.ⓒ 뉴스1/김진희 기자.  같은 날 호찌민시 소재 반한몰 내 CGV 매장에서는 행사 대미를 장식했다.  17일 열리는 '2024 CJ 전국 청소년 태권도 선수권 대회'를 기념해 태권도 선수들을 초청해 격파, 쇼 무대를 꾸며 분위기를 띄웠다. 고객들은 CJ 제품을 맛보면서 태권도라는 K-콘텐츠를 즐겼다.  이와 함께 셰프들이 직접 CJ 제품을 활용한 메뉴를 선보이며 브랜드 홍보 효과를 동시에 누릴 수 있었다.  CJ그룹은 지난 10일부터 9월 1일까지 베트남에서 K컬처 축제 'CJ K FESTA'를 연다.ⓒ 뉴스1/김진희 기자.  CJ는 'CJ K FESTA'를 정례화해 베트남에서 'K-블랙프라이데이'를 구축할 방침이다. 태권도 선수권 대회를 전후로 페스타를 열고 CJ 계열사 5개 브랜드를 대상으로 베트남 전역에서 대규모 할인 행사를 열 계획이다.  이를 통해 베트남 내 CJ 브랜드를 제고하고 CJ가 'K컬처 대표 브랜드'라는 이미지를 각인시킨다는 전략이다.  CJ 관계자는 "CJ가 베트남에 진출한 지 약 25년 됐는데 이제는 CJ 그룹 브랜드를 알리는 차원을 넘어 유기적으로 활용해 시너지를 추구할 것"이라며 "CJ K FESTA를 베트남에서 가장 사랑받는 K컬처 대표 축제로 만들겠다"고 강조했다.  jinny1@news1.kr'''

# 메시지 준비
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

# 프롬프트 생성
prompt = template.safe_substitute({
    "system_content": messages[0]["content"],
    "user_content": messages[1]["content"]
})


In [76]:
prompt = prompt.strip() + '근거:'

In [77]:
print(prompt)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 ESG에 대해서 점수를 계산하는 모델입니다. 평가 기준은 다음과 같습니다.

## 지시사항
주어진 평가 기준을 바탕으로 뉴스 기사로부터 환경, 사회, 지배 구조에 대해서 점수를 측정하십시오.

## 평가 기준
환경
1점: 환경 오염 유발, 법적 제재, 규제 위반 등의 사건이 발생한 경우. 예: 심각한 오염 사고, 규제 위반으로 인한 벌금 부과
2점: 환경 보호에 대한 소극적인 태도 또는 일회성 활동에 그치는 경우. 예: 임시적인 환경 캠페인 참여, 적극적인 감축 목표 미설정
3점: 환경 보호를 위한 노력이 있으나 체계적인 전략이 부족한 경우. 예: 온실가스 감축 계획 발표, 일부 친환경 인증 획득
4점: 체계적인 환경 관리 계획이 있으며, 목표 달성에 꾸준히 투자하는 경우. 예: 지속 가능한 에너지 사용, 체계적인 폐기물 관리 프로그램 운영
5점: 적극적인 환경 정책과 뛰어난 실적 결과를 보이며 업계 선도적 활동을 하는 경우. 예: 탄소 배출 네트 제로(Net Zero) 목표 실현, 친환경 기술 선도

사회
1점: 인권, 근로자 안전 문제 발생, 지역 사회와의 갈등 등 부정적 이슈 발생. 예: 직원 착취 문제, 안전 관리 소홀로 인한 사고
2점: 직원 복지나 지역 사회 기여에 대한 제한적 활동 또는 단기적인 참여. 예: 일회성 기부, 사내 복지 프로그램 부족
3점: 직원 복지 및 사회적 기여에 관심을 가지며 일부 프로그램을 운영하는 경우. 예: 사내 복지 프로그램 운영, 특정 지역 사회 활동 참여
4점: 직원 복지, 다양성, 지역 사회 공헌에 적극적으로 나서며 책임감 있게 운영하는 경우. 예: 다양성 강화 프로그램, 정기적인 사회공헌 활동
5점: 사회적 가치 창출에 지속적으로 기여하며 업계 모범적인 활동을 보여주는 경우. 예: 폭넓은 사회적 책임 활동, 소외 계층 지원 프로그램

지배구조
1점: 지배구조의 투명성 부족, 부패 사건 또는 규제 위반

In [103]:
# 샘플링 파라미터 설정
sampling_params = SamplingParams(
    temperature=0,
    max_tokens=1024,
    stop=["\n<|end_of_text|>", "<|eot_id|>"],  # 문자열로 중지 토큰 지정
    repetition_penalty=1.5
)

# 생성
outputs = llm.generate([prompt], sampling_params)

# 결과 출력
for output in outputs:
    print()
    print(output.outputs[0].text)

Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.18s/it, est. speed input: 1163.55 toks/s, output: 70.63 toks/s]


 이 뉴스는 CJ그룹이 베트남도 문화 콘테스트 ‘CJ K-Festa’를 진행하여 다양한 이벤트를 통해서 자신들의 상품들과 서비스를 넷째 시장 확대로 하고 있다는 내용을 다루고 있습니다. 이는 기업 간 상생 협력을 촉진하려는 긍정적인 움직임이나 관련성을 나타내기 때문에 높은 성능을 볼 가능성이 높습니다. 그러나 특별히 명확하지 않은 부분에서도 여러 분야에서의 영향도를 정확 판단하기 어렵기에 일반적인 평균값을 적용하였으며, 특히 환경(E), 사회(S)에 관한 정보로는 크거나 중요한 영향을 받지는 않음을 확인했습니다. 
ESG:
E: 3
S: 3
G: 4


In [9]:
!wget https://raw.githubusercontent.com/llm-fine-tuning/LLaMA-Factory/refs/heads/main/data/esg_data_test.json

--2024-11-19 06:36:00--  https://raw.githubusercontent.com/llm-fine-tuning/LLaMA-Factory/refs/heads/main/data/esg_data_test.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2806397 (2.7M) [text/plain]
Saving to: ‘esg_data_test.json’

esg_data_test.json  100%[===================>]   2.68M  --.-KB/s    in 0.06s   

2024-11-19 06:36:08 (43.8 MB/s) - ‘esg_data_test.json’ saved [2806397/2806397]



In [10]:
import pandas as pd

df = pd.read_json('esg_data_test.json')

In [11]:
df

,instruction,input,output
0,"""월남쌈 대신 '비비고' 만두 먹고 CGV서 영화 봐요""…'베트남 젠지' 홀린 CJ...",,근거: 이 뉴스 기사는 CJ그룹이 베트남에서 K컬처 축제 'CJ K FESTA'를 ...
1,"KDI ""한국경제 부진 지속…수출 위축에 내수도 둔화"" 국책연구원인 한국개발연구원,...",,근거: 해당 뉴스 기사는 KDI가 한국 경제의 경기 부진과 수출 위축 및 내수 둔화...
2,개성 가득 이색 팝업 ’글로스튜디오’ 체험하는 배우 이이경 (서울=뉴스1) = BA...,,근거: 이 뉴스 기사는 BAT로스만스의 이색 팝업 체험 공간인 '글로 스튜디오' 오...
3,"삼성전자 노조, 사측에 화해 손짓…""파업 상황 조속 해결"" 전국삼성전자노동조합 조합...",,근거: 이 뉴스 기사는 삼성전자와 전국삼성전자노동조합 간의 임금 및 성과금 관련 교...
4,"‘클라우드’ 돕는 트레이너…‘AI’까지 넘본다 [천억클럽] 인공지능(AI) 시대, ...",,근거: 이 뉴스 기사에서는 베스핀글로벌이라는 클라우드 스타트업의 성장과 사업 전략에...
...,...,...,...
927,부산시 전기차 구매 보조금 지원대상 확대 [KBS 부산]부산시가 올해 상반기 전기자...,,근거: 이 뉴스 기사는 부산시가 전기자동차 구매 보조금 지원 대상을 확대하고 있는 ...
928,KT&G “세계적인 의결권자문사의 이사회 제안 안건 ‘찬성 권고’ 대환영” 글로벌 ...,,근거: 이 뉴스 기사는 KT&G 이사회 제안이 글로벌 의결권자문사 글래스루이스의 지...
929,서산 농작물 창고서 불…2백만 원 피해 [KBS 대전]오늘(27일) 오전 10시쯤 ...,,근거: 이 뉴스 기사는 창고에서 발생한 화재로 인한 피해 내용에 대해 다루고 있습니...
930,"현대차, 신형 코나 전기차 모델 첫 공개 자동요약 기사 제목과 주요 문장을 기반으로...",,근거: 현대자동차는 새로운 전기차 모델 '디 올 뉴 코나 일렉트릭'을 공개하여 전기...


In [12]:
inputs = df['instruction'].to_list()

In [13]:
print(inputs[0])

"월남쌈 대신 '비비고' 만두 먹고 CGV서 영화 봐요"…'베트남 젠지' 홀린 CJ CJ그룹은 지난 10일부터 9월 1일까지 베트남에서 K컬처 축제 'CJ K FESTA'를 연다.ⓒ 뉴스1/김진희 기자.  (호찌민=뉴스1) 김진희 기자 = "CJ(001040) 로고가 화려하고 예쁩니다. CGV를 통해 CJ를 알게 됐는데 젊고 역동적이어서 '젠지' 세대에 적합한 이미지라는 생각이 듭니다." -화이빠오(24·남)  지난 15~16일(현지시간) 베트남 호찌민 일대에서 열린 'CJ K FESTA'를 찾은 베트남 현지인들은 극찬을 쏟아냈다.  CJ그룹은 베트남에 진출한 계열사를 총동원해 CJ K FESTA를 올해 처음으로 개최했다. 이번 축제는 △K푸드 Week(8월 10~16일) △K스포츠 Week(8월 17~23일) △K무비 Week(8월 24일~9월 1일) 등 3가지 테마로 구성됐다.  CJ는 호찌민 소재 유명 마트와 쇼핑몰 내 CGV 매장에서 푸드쇼를 운영하며 브랜드 홍보에 적극 나섰다.  CJ그룹은 지난 10일부터 9월 1일까지 베트남에서 K컬처 축제 'CJ K FESTA'를 연다.ⓒ 뉴스1/김진희 기자.  15일 오후 호찌민 소재 메가마켓에는 더운 날씨임에도 마트 외부 공간에 사람들로 북적였다. CJ 제품을 선보이는 푸드트럭을 구경하기 위해 찾은 이들이었다. 한국과 달리 베트남에서는 푸드트럭이 생소하다고 한다.  이날 CJ는 CJ제일제당 비비고의 해산물 만두, 호떡, 김스낵과 뚜레쥬르의 빵을 선보였다. CJ제일제당의 자회사인 베트남 현지 냉동식품기업 '까우제'(Cautre)의 제품도 만나볼 수 있었다.  호찌민 시민 화이빠오는 "예전에 만두를 많이 먹어 봤지만 비비고 만두는 독특하고 맛있다"며 "특히 푸드트럭을 처음 봐서 신기한데, 이렇게 화려하게 시식·광고하는 건 처음 본다"고 말했다.  CJ그룹은 지난 10일부터 9월 1일까지 베트남에서 K컬처 축제 'CJ K FESTA'를 연다.ⓒ 뉴스1/김진희 기자.  아이와 함께 푸드트럭을 방문한 주부 흐엉(34·여

In [38]:
def make_prompt(system_prompt, news_article):
    # 프롬프트 템플릿 준비
    template = string.Template("""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    
${system_content}<|eot_id|><|start_header_id|>user<|end_header_id|>
    
${user_content}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
""")
    
    user_prompt = news_article
    
    # 메시지 준비
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
    
    # 프롬프트 생성
    prompt = template.safe_substitute({
        "system_content": messages[0]["content"],
        "user_content": messages[1]["content"]
    })

    prompt = prompt.strip() + "근거:"
    return prompt

In [39]:
prompt = make_prompt(system_prompt, inputs[0])
print(prompt)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    
당신은 ESG에 대해서 점수를 계산하는 모델입니다. 평가 기준은 다음과 같습니다.

## 지시사항
주어진 평가 기준을 바탕으로 뉴스 기사로부터 환경, 사회, 지배 구조에 대해서 점수를 측정하십시오.

## 평가 기준
환경
1점: 환경 오염 유발, 법적 제재, 규제 위반 등의 사건이 발생한 경우. 예: 심각한 오염 사고, 규제 위반으로 인한 벌금 부과
2점: 환경 보호에 대한 소극적인 태도 또는 일회성 활동에 그치는 경우. 예: 임시적인 환경 캠페인 참여, 적극적인 감축 목표 미설정
3점: 환경 보호를 위한 노력이 있으나 체계적인 전략이 부족한 경우. 예: 온실가스 감축 계획 발표, 일부 친환경 인증 획득
4점: 체계적인 환경 관리 계획이 있으며, 목표 달성에 꾸준히 투자하는 경우. 예: 지속 가능한 에너지 사용, 체계적인 폐기물 관리 프로그램 운영
5점: 적극적인 환경 정책과 뛰어난 실적 결과를 보이며 업계 선도적 활동을 하는 경우. 예: 탄소 배출 네트 제로(Net Zero) 목표 실현, 친환경 기술 선도

사회
1점: 인권, 근로자 안전 문제 발생, 지역 사회와의 갈등 등 부정적 이슈 발생. 예: 직원 착취 문제, 안전 관리 소홀로 인한 사고
2점: 직원 복지나 지역 사회 기여에 대한 제한적 활동 또는 단기적인 참여. 예: 일회성 기부, 사내 복지 프로그램 부족
3점: 직원 복지 및 사회적 기여에 관심을 가지며 일부 프로그램을 운영하는 경우. 예: 사내 복지 프로그램 운영, 특정 지역 사회 활동 참여
4점: 직원 복지, 다양성, 지역 사회 공헌에 적극적으로 나서며 책임감 있게 운영하는 경우. 예: 다양성 강화 프로그램, 정기적인 사회공헌 활동
5점: 사회적 가치 창출에 지속적으로 기여하며 업계 모범적인 활동을 보여주는 경우. 예: 폭넓은 사회적 책임 활동, 소외 계층 지원 프로그램

지배구조
1점: 지배구조의 투명성 부족, 부패 사건 또는 규

In [80]:
prompts = []
for input in inputs:
    prompts.append(make_prompt(system_prompt, input))

In [81]:
len(prompts)

932

## 하나의 데이터에 대한 호출

In [82]:
results = llm.generate([prompts[0]], sampling_params)
print(results[0].outputs[0].text)

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.71s/it, est. speed input: 1487.92 toks/s, output: 68.59 toks/s]

 이 뉴스 기사는 CJ그룹이 베트남에서 K컬처 축제 'CJ K FESTA'를 개최하여 다양한 K-콘텐츠를 홍보하는 내용을 다루고 있습니다. 환경에 대한 명시적 내용은 없으며, 사회적 측면에서는 지역 사회와의 긍정적인 상호 작용을 통해 문화 교류를 촉진하고 있습니다. 지배구조와 관련된 정보는 포함되어 있지 않습니다.
E: 2
S: 4
G: 3


## 다수의 데이터에 대한 호출

In [83]:
results = llm.generate(prompts[:5], sampling_params)

Processed prompts: 100%|██████████| 5/5 [00:03<00:00,  1.62it/s, est. speed input: 3234.65 toks/s, output: 183.79 toks/s]


In [84]:
for result in results:
    print(result.outputs[0].text)
    print('--')

 이 뉴스 기사는 CJ그룹이 베트남에서 K컬처 축제 'CJ K FESTA'를 개최하여 브랜드 홍보를 강화하는 내용을 다루고 있습니다. 환경에 대한 명시적 내용은 없으며, 사회적 측면에서는 지역 사회와의 긍정적인 상호 작용을 통해 문화 교류를 촉진하고 있습니다. 지배구조와 관련된 정보는 포함되어 있지 않습니다.
E: 2
S: 4
G: 3
--
 이 뉴스 기사는 한국 경제의 부진과 관련된 내용을 다루고 있습니다. 환경, 사회, 지배구조와 직접적으로 관련된 정보는 포함되어 있지 않으며, 경제 동향에 대한 분석과 예측에 초점이 맞춰져 있습니다. 따라서 ESG 평가 기준에 따라 환경, 사회, 지배구조에 대한 점수를 부여하기 어렵습니다.
E: 3
S: 3
G: 3
--
 이 뉴스 기사는 '글로 스튜디오'라는 팝업 체험 공간의 오픈 소식을 다루고 있습니다. 환경, 사회, 지배구조와 관련된 명시적인 내용은 없습니다. 따라서 ESG 평가에 필요한 정보가 부족합니다.
E: 3
S: 3
G: 3
--
 이 뉴스 기사는 삼성전자 노조와 사측 간의 파업 상황과 대화 재개 가능성을 다루고 있습니다. 환경에 대한 명시적 내용은 없으며, 사회적 측면에서는 노사 간의 갈등과 파업으로 인한 사회적 불안이 드러나고 있습니다. 지배구조 측면에서는 노사 간의 소통과 협상이 이루어지고 있어 투명성과 협력의 가능성이 엿보입니다.
E: 2
S: 2
G: 3
--
 이 뉴스 기사는 클라우드 서비스 제공업체인 베스핀글로벌의 성장과 AI 분야에서의 확장을 다루고 있습니다. 환경에 대한 직접적인 언급은 없으며, 사회적 측면에서는 클라우드 서비스를 통해 기업들이 데이터 관리와 AI 활용을 돕는다는 점에서 긍정적인 기여를 하고 있다고 볼 수 있습니다. 지배구조 측면에서는 전문 인력 양성과 클라우드 자격증 취득을 통해 경쟁력을 강화하고 있으며, 이는 투명하고 윤리적인 지배구조를 유지하려는 노력으로 해석될 수 있습니다.
E: 3
S: 4
G: 4
--


## 평가를 위한 테스트 데이터 전체에 대한 호출

In [85]:
results = llm.generate(prompts, sampling_params)

Processed prompts: 100%|██████████| 932/932 [03:11<00:00,  4.87it/s, est. speed input: 7449.05 toks/s, output: 636.24 toks/s]


In [95]:
preds = []
for result in results:
    preds.append(result.outputs[0].text)

In [96]:
len(preds)

932

In [97]:
print(preds[0])

 이 뉴스 기사는 CJ그룹이 베트남에서 K컬처 축제 'CJ K FESTA'를 개최하여 브랜드 홍보를 강화하는 내용을 다루고 있습니다. 환경에 대한 명시적 내용은 없으며, 사회적 측면에서는 지역 사회와의 긍정적인 상호 작용을 통해 문화 교류를 촉진하고 있습니다. 지배구조와 관련된 정보는 포함되어 있지 않습니다.
E: 2
S: 4
G: 3


In [122]:
import re
import numpy as np
import pandas as pd

def extract_scores(text):
    """
    Extract ESG scores from a given string.
    
    Parameters:
        text (str): Input string containing "E: x", "S: x", "G: x".
    
    Returns:
        dict: A dictionary with keys 'E', 'S', 'G' and corresponding integer scores.
              Returns None if extraction fails.
    """
    pattern = r'E: (\d+)\s+S: (\d+)\s+G: (\d+)'
    try:
        match = re.search(pattern, text)
        if match:
            return {'E': int(match.group(1)), 'S': int(match.group(2)), 'G': int(match.group(3))}
        else:
            raise ValueError("ESG scores not found in the text.")
    except Exception as e:
        print(f"[Error] Failed to extract scores: {e}")
        return None

def calculate_metrics(predictions, ground_truths):
    """
    Calculate MAE and MSE for multiple predictions and ground truths.
    Assign maximum error for invalid predictions.
    
    Parameters:
        predictions (list of str): List of prediction strings containing ESG scores.
        ground_truths (list of str): List of ground truth strings containing ESG scores.
    
    Returns:
        dict: A dictionary containing 'MAE' and 'MSE'.
    """
    pred_scores = [extract_scores(pred) for pred in predictions]
    true_scores = [extract_scores(gt) for gt in ground_truths]

    adjusted_preds = []
    adjusted_trues = []
    for i, (pred, true) in enumerate(zip(pred_scores, true_scores)):
        # If prediction is invalid
        if pred is None:
            print(f"[Info] Prediction {i} is invalid. Applying maximum error.")
            pred = {'E': 1, 'S': 1, 'G': 1}  # Maximum error predictions
        # If ground truth is invalid
        if true is None:
            print(f"[Info] Ground truth {i} is invalid. Using default values.")
            true = {'E': 5, 'S': 5, 'G': 5}  # Default values for invalid ground truths
        adjusted_preds.append(pred)
        adjusted_trues.append(true)

    # Convert to DataFrame for easier processing
    pred_df = pd.DataFrame(adjusted_preds)
    true_df = pd.DataFrame(adjusted_trues)

    # Calculate absolute and squared errors
    abs_errors = np.abs(pred_df - true_df)
    sq_errors = (pred_df - true_df) ** 2

    # Calculate MAE and MSE
    mae = abs_errors.mean().mean()
    mse = sq_errors.mean().mean()

    return {'MAE': mae, 'MSE': mse}

# Example data
predictions = [
    '''근거: 이 뉴스 기사는 CJ그룹이 베트남에서 K컬처 축제 'CJ K FESTA'를 개최하여 브랜드 홍보를 강화하는 내용을 다루고 있습니다. 환경에 대한 명시적 내용은 없으며, 사회적 측면에서는 지역 사회와의 긍정적인 상호작용을 통해 브랜드 인지도를 높이는 활동을 하고 있습니다. 지배구조와 관련된 내용은 명시적으로 다루어지지 않았습니다. 
    E: 2
    S: 3
    G: 3''',
    '''근거: 잘못된 형식의 데이터입니다.'''
]

ground_truths = [
    '''근거: 이 뉴스 기사는 CJ그룹이 베트남에서 K컬처 축제 'CJ K FESTA'를 개최하고, 베트남 시장에서 브랜드 홍보를 적극적으로 진행하는 내용을 다루고 있습니다. 환경에 대한 직접적인 내용은 없으며, 주로 기업의 사회적 기여와 브랜드 이미지 형성에 관련된 내용이 언급됩니다. 이 행사는 지역 사회를 대상으로 한국 문화를 홍보하고, 다양한 사회적 참여를 통해 긍정적인 이미지를 구축하려는 노력이 보입니다. 지배구조에 대한 구체적인 내용은 기사에 포함되어 있지 않습니다.
    E: 2
    S: 3
    G: 3''',
    '''근거: 이 뉴스도 잘못된 형식입니다.'''
]

# Calculate metrics
metrics = calculate_metrics(predictions, ground_truths)
print(f"MAE: {metrics['MAE']}")
print(f"MSE: {metrics['MSE']}")

[Error] Failed to extract scores: ESG scores not found in the text.
[Error] Failed to extract scores: ESG scores not found in the text.
[Info] Prediction 1 is invalid. Applying maximum error.
[Info] Ground truth 1 is invalid. Using default values.
MAE: 2.0
MSE: 8.0


Ground truth 문제 처리:
- 정답(Ground truth)에 문제가 있을 경우 [Info] 메시지를 출력합니다.
- 기본값({E: 5, S: 5, G: 5})으로 설정합니다.

Prediction 문제 처리:
- 예측값(Prediction)에 문제가 있을 경우 최대 오차를 적용합니다.
- 정답과 관계없이 무조건 E: 1, S: 1, G: 1로 설정합니다.

In [99]:
preds = ['근거:' + pred for pred in preds]

In [111]:
print(preds[0])

근거: 이 뉴스 기사는 CJ그룹이 베트남에서 K컬처 축제 'CJ K FESTA'를 개최하여 브랜드 홍보를 강화하는 내용을 다루고 있습니다. 환경에 대한 명시적 내용은 없으며, 사회적 측면에서는 지역 사회와의 긍정적인 상호 작용을 통해 문화 교류를 촉진하고 있습니다. 지배구조와 관련된 정보는 포함되어 있지 않습니다.
E: 2
S: 4
G: 3


In [112]:
print(df['output'].loc[0])

근거: 이 뉴스 기사는 CJ그룹이 베트남에서 K컬처 축제 'CJ K FESTA'를 개최하고, 베트남 시장에서 브랜드 홍보를 적극적으로 진행하는 내용을 다루고 있습니다. 환경에 대한 직접적인 내용은 없으며, 주로 기업의 사회적 기여와 브랜드 이미지 형성에 관련된 내용이 언급됩니다. 이 행사는 지역 사회를 대상으로 한국 문화를 홍보하고, 다양한 사회적 참여를 통해 긍정적인 이미지를 구축하려는 노력이 보입니다. 지배구조에 대한 구체적인 내용은 기사에 포함되어 있지 않습니다.
E: 2
S: 4
G: 3


In [113]:
print(preds[1])

근거: 이 뉴스 기사는 한국 경제의 부진과 관련된 내용을 다루고 있습니다. 환경, 사회, 지배구조와 직접적으로 관련된 정보는 포함되어 있지 않으며, 경제 동향에 대한 분석과 예측에 초점이 맞춰져 있습니다. 따라서 ESG 평가 기준에 따라 환경, 사회, 지배구조에 대한 점수를 부여하기 어렵습니다.
E: 3
S: 3
G: 3


In [114]:
print(df['output'].loc[1])

근거: 해당 뉴스 기사는 KDI가 한국 경제의 경기 부진과 수출 위축 및 내수 둔화를 진단하는 내용을 다루고 있습니다. 환경, 사회, 지배구조와 관련된 구체적인 내용은 포함되어 있지 않으며, 경제 전반의 상황에 대한 분석에 중점을 두고 있습니다.
E: 3
S: 3
G: 3


In [118]:
# Example data
predictions = preds
ground_truths = df['output'].to_list()

# Calculate metrics
metrics = calculate_metrics(predictions, ground_truths)
print(f"MAE: {metrics['MAE']}")
print(f"MSE: {metrics['MSE']}")

[Error] Failed to extract scores: ESG scores not found in the text.
[Error] Failed to extract scores: ESG scores not found in the text.
[Error] Failed to extract scores: ESG scores not found in the text.
[Error] Failed to extract scores: ESG scores not found in the text.
[Error] Failed to extract scores: ESG scores not found in the text.
[Error] Failed to extract scores: ESG scores not found in the text.
[Error] Failed to extract scores: ESG scores not found in the text.
[Error] Failed to extract scores: ESG scores not found in the text.
[Info] Prediction 15 is invalid. Applying maximum error.
[Info] Ground truth 20 is invalid. Using default values.
[Info] Prediction 60 is invalid. Applying maximum error.
[Info] Ground truth 109 is invalid. Using default values.
[Info] Ground truth 138 is invalid. Using default values.
[Info] Ground truth 187 is invalid. Using default values.
[Info] Ground truth 786 is invalid. Using default values.
[Info] Prediction 888 is invalid. Applying maximum er